In [1]:
pip install strawberryfields



[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np

# set the random seed
np.random.seed(42)

# import Strawberry Fields
import strawberryfields as sf
from strawberryfields.ops import *

# initialize a 4 mode program
boson_sampling = sf.Program(4)

with boson_sampling.context as q:
    # prepare the input fock states
    Fock(1) | q[0]
    Fock(1) | q[1]
    Vac     | q[2]
    Fock(1) | q[3]

    # rotation gates
    #Rgate(0.5719)  | q[0]
    #Rgate(-1.9782) | q[1]
    #Rgate(2.0603)  | q[2]
    #Rgate(0.0644)  | q[3]
    # rotation gates
    Rgate(0.0644)  | q[0]
    Rgate(0.5719) | q[1]
    Rgate(2.0603)  | q[2]
    Rgate(-1.9782)  | q[3]


    # beamsplitter array
    #BSgate(0.7804, 0.8578)  | (q[0], q[1])
    #BSgate(0.06406, 0.5165) | (q[2], q[3])
    #BSgate(0.473, 0.1176)   | (q[1], q[2])
    #BSgate(0.563, 0.1517)   | (q[0], q[1])
    #BSgate(0.1323, 0.9946)  | (q[2], q[3])
    #BSgate(0.311, 0.3231)   | (q[1], q[2])
    #BSgate(0.4348, 0.0798)  | (q[0], q[1])
    #BSgate(0.4368, 0.6157)  | (q[2], q[3])
    # beamsplitter array
    BSgate(0.7804, 0.8578)  | (q[0], q[1])
    BSgate(0.06406, 0.5165) | (q[2], q[3])
    BSgate(0.7804, 0.8578)   | (q[1], q[2])
    BSgate(0.06406, 0.5165)   | (q[0], q[1])
    BSgate(0.473, 0.1176)  | (q[2], q[3])
    BSgate(0.563, 0.1517)   | (q[1], q[2])
    BSgate(0.1323, 0.9946)  | (q[0], q[1])
    BSgate(0.311, 0.3231)  | (q[2], q[3])
    

2022-08-23 09:31:59.583896: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-23 09:31:59.583929: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
#MeasureFock() | q

In [4]:
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": 7})

In [5]:
results = eng.run(boson_sampling)

In [6]:
print(results)

<Result: shots=0, num_modes=0, contains state=True>


In [7]:
probs = results.state.all_fock_probs()

In [8]:
print(probs[1, 1, 0, 1])
print(probs[2, 0, 0, 1])

0.02701656565980743
0.2280016292105849


In [9]:
import numpy as np
from numpy.linalg import multi_dot
from scipy.linalg import block_diag

In [10]:
Uphase = np.diag([np.exp(0.5719*1j),np.exp(-1.9782*1j),np.exp(2.0603*1j),np.exp(0.0644*1j)])

In [11]:
BSargs = [
    (0.7804, 0.8578),
    (0.06406, 0.5165),
    (0.473, 0.1176),
    (0.563, 0.1517),
    (0.1323, 0.9946),
    (0.311, 0.3231),
    (0.4348, 0.0798),
    (0.4368, 0.6157)
]

In [12]:
t_r_amplitudes = [(np.cos(q), np.exp(p*1j)*np.sin(q)) for q,p in BSargs]

In [13]:
BSunitaries = [np.array([[t, -np.conj(r)], [r, t]]) for t,r in t_r_amplitudes]

In [14]:
UBS1 = block_diag(*BSunitaries[0:2])
UBS2 = block_diag([[1]], BSunitaries[2], [[1]])
UBS3 = block_diag(*BSunitaries[3:5])
UBS4 = block_diag([[1]], BSunitaries[5], [[1]])
UBS5 = block_diag(*BSunitaries[6:8])

In [15]:
U = multi_dot([UBS5, UBS4, UBS3, UBS2, UBS1, Uphase])
print(np.round(U, 4))

[[ 0.2195-0.2565j  0.6111+0.5242j -0.1027+0.4745j -0.0273+0.0373j]
 [ 0.4513+0.6026j  0.457 +0.0123j  0.1316-0.4504j  0.0353-0.0532j]
 [ 0.0387+0.4927j -0.0192-0.3218j -0.2408+0.5244j -0.4584+0.3296j]
 [-0.1566+0.2246j  0.11  -0.1638j -0.4212+0.1836j  0.8188+0.068j ]]


In [16]:
prog_unitary = sf.Program(4)
prog_unitary.circuit = boson_sampling.circuit[4:]
prog_compiled = prog_unitary.compile(compiler="gaussian_unitary")

In [17]:
prog_compiled.print()

GaussianTransform([[ 0.6372 -0.7173  0.1376  0.0383 -0.0572 -0.2229 -0.0776  0.0112]
 [ 0.343   0.1781 -0.1118 -0.206  -0.1989  0.0694  0.8458  0.1897]
 [ 0.0646  0.1974 -0.2586  0.3623 -0.5164 -0.4502  0.0224 -0.5375]
 [-0.109   0.0198 -0.3207 -0.2984 -0.3883 -0.3966 -0.2775  0.6409]
 [ 0.0572  0.2229  0.0776 -0.0112  0.6372 -0.7173  0.1376  0.0383]
 [ 0.1989 -0.0694 -0.8458 -0.1897  0.343   0.1781 -0.1118 -0.206 ]
 [ 0.5164  0.4502 -0.0224  0.5375  0.0646  0.1974 -0.2586  0.3623]
 [ 0.3883  0.3966  0.2775 -0.6409 -0.109   0.0198 -0.3207 -0.2984]]) | (q[0], q[1], q[2], q[3])


In [18]:
S = prog_compiled.circuit[0].op.p[0]
U = S[:4, :4] + 1j*S[4:, :4]
print(U)

[[ 0.6372+0.0572j -0.7173+0.2229j  0.1376+0.0776j  0.0383-0.0112j]
 [ 0.343 +0.1989j  0.1781-0.0694j -0.1118-0.8458j -0.206 -0.1897j]
 [ 0.0646+0.5164j  0.1974+0.4502j -0.2586-0.0224j  0.3623+0.5375j]
 [-0.109 +0.3883j  0.0198+0.3966j -0.3207+0.2775j -0.2984-0.6409j]]


In [19]:
boson_sampling = sf.Program(4)

with boson_sampling.context as q:
    # prepare the input fock states
    Fock(1) | q[0]
    Fock(1) | q[1]
    Vac     | q[2]
    Fock(1) | q[3]

    Interferometer(U) | q

In [20]:
boson_sampling.compile(compiler="fock").print()

Fock(1) | (q[0])
Fock(1) | (q[1])
Vac | (q[2])
Fock(1) | (q[3])
Rgate(0.3237) | (q[0])
BSgate(0.7932, 0) | (q[0], q[1])
Rgate(-2.344) | (q[2])
BSgate(0.2473, 0) | (q[2], q[3])
Rgate(2.724) | (q[1])
BSgate(1.123, 0) | (q[1], q[2])
Rgate(-3.129) | (q[0])
BSgate(0.1817, 0) | (q[0], q[1])
Rgate(2.858) | (q[0])
Rgate(0.4223) | (q[1])
Rgate(1.939) | (q[2])
Rgate(4.34) | (q[3])
BSgate(-0.6788, 0) | (q[2], q[3])
Rgate(2.82) | (q[2])
BSgate(-0.1073, 0) | (q[1], q[2])
Rgate(-2.951) | (q[1])


In [21]:
print(probs[2,0,0,1])

0.2280016292105849


In [22]:
from thewalrus import perm

In [23]:
U[:,[0,1,3]]

array([[ 0.6372+0.0572j, -0.7173+0.2229j,  0.0383-0.0112j],
       [ 0.343 +0.1989j,  0.1781-0.0694j, -0.206 -0.1897j],
       [ 0.0646+0.5164j,  0.1974+0.4502j,  0.3623+0.5375j],
       [-0.109 +0.3883j,  0.0198+0.3966j, -0.2984-0.6409j]])

In [24]:
U[:,[0,1,3]][[0,0,3]]

array([[ 0.6372+0.0572j, -0.7173+0.2229j,  0.0383-0.0112j],
       [ 0.6372+0.0572j, -0.7173+0.2229j,  0.0383-0.0112j],
       [-0.109 +0.3883j,  0.0198+0.3966j, -0.2984-0.6409j]])

In [25]:
print(np.abs(perm(U[:, [0,1,3]][[0,0,3]]))**2 / 2)

0.2280016292105848


In [26]:
BS = np.abs(perm(U[:, [0,1,3]][[0,0,3]]))**2 / 2
SF = probs[2,0,0,1]

print(100*np.abs(BS-SF)/BS)

3.652023326990382e-14


In [27]:
print(probs[3,0,0,0])
print(np.abs(perm(U[:, [0,1,3]][[0,0,0]]))**2 / 6)

0.002202343953200861
0.00220234395320086


In [28]:
print(probs[1,1,0,1])
print(np.abs(perm(U[:, [0,1,3]][[0,1,3]]))**2 / 1)

0.02701656565980743
0.02701656565980743
